In [2]:
#import libraries

import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity
import json

In [4]:
#import data

df = pd.read_csv('../assets/electronics.csv', delimiter='\t')
df.head(10)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,41409413,R2MTG1GCZLR2DK,B00428R89M,112201306,yoomall 5M Antenna WIFI RP-SMA Female to Male ...,Electronics,5,0,0,N,Y,Five Stars,As described.,2015-08-31
1,US,49668221,R2HBOEM8LE9928,B000068O48,734576678,"Hosa GPM-103 3.5mm TRS to 1/4"" TRS Adaptor",Electronics,5,0,0,N,Y,It works as advertising.,It works as advertising.,2015-08-31
2,US,12338275,R1P4RW1R9FDPEE,B000GGKOG8,614448099,Channel Master Titan 2 Antenna Preamplifier,Electronics,5,1,1,N,Y,Five Stars,Works pissa,2015-08-31
3,US,38487968,R1EBPM82ENI67M,B000NU4OTA,72265257,LIMTECH Wall charger + USB Hotsync & Charging ...,Electronics,1,0,0,N,Y,One Star,Did not work at all.,2015-08-31
4,US,23732619,R372S58V6D11AT,B00JOQIO6S,308169188,Skullcandy Air Raid Portable Bluetooth Speaker,Electronics,5,1,1,N,Y,Overall pleased with the item,Works well. Bass is somewhat lacking but is pr...,2015-08-31
5,US,21257820,R1A4514XOYI1PD,B008NCD2LG,976385982,Pioneer SP-BS22-LR Andrew Jones Designed Books...,Electronics,5,1,1,N,Y,Five Stars,The quality on these speakers is insanely good...,2015-08-31
6,US,3084991,R20D9EHB7N20V6,B00007FGUF,670878953,C2G/Cables to Go 03170 3.5mm F/F Stereo Coupler,Electronics,5,0,0,N,Y,Lifesaver,Wish I could give this product more than five ...,2015-08-31
7,US,8153674,R1WUTD8MVSROJU,B00M9V2RMM,508452933,COOLEAD-HDMI Switcher BOX,Electronics,5,0,0,N,Y,Five Stars,works great,2015-08-31
8,US,52246189,R1QCYLT25812DM,B00J3O9DYI,766372886,Philips Wireless Portable Speaker,Electronics,4,0,0,N,Y,Four Stars,Great sound and compact. Battery life seems go...,2015-08-31
9,US,41463864,R904DQPBCEM7A,B00NS1A0E4,458130381,PlayStation 3 3D Glasses (Super Value 4 Pack),Electronics,4,0,0,N,Y,Four Stars,It works well~~~,2015-08-31


# Dataset exploration and cleaning

In [3]:
df.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

In [4]:
# Drop columns

In [5]:
re = df.drop(['marketplace',  'review_id', 'product_id',
       'product_parent',  'product_category',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'], axis=1)

In [6]:
re

,customer_id,product_title,star_rating
0,41409413,yoomall 5M Antenna WIFI RP-SMA Female to Male ...,5
1,49668221,"Hosa GPM-103 3.5mm TRS to 1/4"" TRS Adaptor",5
2,12338275,Channel Master Titan 2 Antenna Preamplifier,5
3,38487968,LIMTECH Wall charger + USB Hotsync & Charging ...,1
4,23732619,Skullcandy Air Raid Portable Bluetooth Speaker,5
...,...,...,...
50994,16993644,8 pcs of Tenergy C Size 5000mAh High Capacity ...,5
50995,12487129,VAlinks(TM) 5 FT 1.5m HDMI Male to 3RCA Male E...,5
50996,2071506,SunJack USB Battery Charger for Rechargeable A...,5
50997,52016360,BellO 60 in. TV Stand - Espresso,5


In [7]:
re.isnull().sum()

customer_id      0
product_title    0
star_rating      0
dtype: int64

In [8]:
re.shape

(50999, 3)

In [9]:
re.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50999 entries, 0 to 50998
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_id    50999 non-null  int64 
 1   product_title  50999 non-null  object
 2   star_rating    50999 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ MB


In [10]:
df['star_rating'].value_counts()

5    31333
4     7571
1     5852
3     3508
2     2735
Name: star_rating, dtype: int64

# Recommender System

- Based on recommended featues
- Based on collaborative user based -ratings
- Based on collaborative user based -similar purchases
- Based on collaborative user based -ratings/similar purchases

# item-based collaborative recommender puchases/ratings - content item based

In [11]:
pivot = pd.pivot_table(re, index='product_title', columns='customer_id', values='star_rating')
pivot

customer_id,10922,12683,14292,14435,15844,15937,17106,21238,22000,23251,...,53090413,53090839,53090971,53091649,53091792,53092785,53093913,53093948,53094830,53096363
product_title,,,,,,,,,,,,,,,,,,,,,
"#1 BEST SELLING Waterproof Bluetooth Speaker with Powerbank, EVIZIAN® ROKON CHARGE 2 in 1 Ultra Portable Wireless Waterproof Shockproof Dust Proof Rugged Bluetooth Speaker with Built In Powerbank - Better Sound, Better Volume, Incredible Online Price - The Perfect Indoor Outdoor Speaker.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"#1 Universal TV Box Mount Holder Mounting Bracket Shelf For Apple TV 2/3, Roku TV 1/2/3, MXQ, MX 3, ONVO Matricom, SkyStreamX 4 Android TV Box, CHOETECH 6-Port, Photive 6-Port, Anker 6-Port/5-Port/2-Port, Poweradd 6-Port/5-Port Desktop USB Charger& More- Retractable Clip Design Patent Pending",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#364 Renata Watch Batteries 2Pcs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#394 Renata Watch Batteries 2Pcs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
%Original Superlux HD668B Semi-open Dynamic Stereo Professional Studio Standard Monitoring Headphones For DJ Music Detachable Audio Cable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xDuoo TA-03 XMOS WM8740 24Bit/192KHz USB DAC 12AU7 Tube Headphone Amplifier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
yoomall 5M Antenna WIFI RP-SMA Female to Male Extensionl Cable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zebest okcharger902 House Wall Power Charger Cord for Barnes and Noble Nook,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
pivot.shape

(15961, 44031)

# Sparse matrix

In [13]:
sparse_pivot = sparse.csr_matrix(pivot.fillna(0))
print(sparse_pivot)

  (0, 4436)	5.0
  (0, 11260)	5.0
  (0, 14918)	5.0
  (0, 19744)	5.0
  (0, 19919)	4.0
  (0, 22462)	5.0
  (0, 26359)	5.0
  (0, 26477)	4.0
  (0, 33401)	5.0
  (0, 34649)	5.0
  (0, 39368)	5.0
  (0, 41697)	5.0
  (0, 41888)	4.0
  (1, 31244)	5.0
  (2, 16528)	4.0
  (3, 4674)	5.0
  (3, 12466)	1.0
  (4, 3355)	5.0
  (4, 4528)	3.0
  (4, 4931)	5.0
  (4, 5626)	5.0
  (4, 6291)	5.0
  (4, 7141)	4.0
  (4, 7558)	3.0
  (4, 12878)	3.0
  :	:
  (15959, 469)	4.0
  (15959, 2397)	2.0
  (15959, 4817)	4.0
  (15959, 6665)	5.0
  (15959, 6689)	5.0
  (15959, 6737)	5.0
  (15959, 6787)	5.0
  (15959, 6837)	5.0
  (15959, 7989)	3.0
  (15959, 12147)	5.0
  (15959, 15388)	5.0
  (15959, 17809)	5.0
  (15959, 18888)	5.0
  (15959, 19899)	5.0
  (15959, 19912)	5.0
  (15959, 21999)	5.0
  (15959, 25226)	3.0
  (15959, 32997)	5.0
  (15959, 39713)	5.0
  (15959, 43764)	2.0
  (15960, 5203)	4.0
  (15960, 5286)	5.0
  (15960, 6558)	5.0
  (15960, 9605)	5.0
  (15960, 33437)	5.0


In [14]:
sparse_pivot.shape

(15961, 44031)

# calculate Pairwise Distance

In [15]:
recommender = pairwise_distances(sparse_pivot)


In [16]:
#create distance dataframe
recommender_df = pd.DataFrame(recommender, columns=pivot.index, index=pivot.index)
recommender_df

product_title,"#1 BEST SELLING Waterproof Bluetooth Speaker with Powerbank, EVIZIAN® ROKON CHARGE 2 in 1 Ultra Portable Wireless Waterproof Shockproof Dust Proof Rugged Bluetooth Speaker with Built In Powerbank - Better Sound, Better Volume, Incredible Online Price - The Perfect Indoor Outdoor Speaker.","#1 Universal TV Box Mount Holder Mounting Bracket Shelf For Apple TV 2/3, Roku TV 1/2/3, MXQ, MX 3, ONVO Matricom, SkyStreamX 4 Android TV Box, CHOETECH 6-Port, Photive 6-Port, Anker 6-Port/5-Port/2-Port, Poweradd 6-Port/5-Port Desktop USB Charger& More- Retractable Clip Design Patent Pending",#364 Renata Watch Batteries 2Pcs,#394 Renata Watch Batteries 2Pcs,%Original Superlux HD668B Semi-open Dynamic Stereo Professional Studio Standard Monitoring Headphones For DJ Music Detachable Audio Cable,(1) CD / DVD Binder - 272 Capacity - Black- #CRY-CDH272BK,(1) Magazine Stor-Folio - 2.5 Inch Storage Box - BCW,(10) 3v Battery CR1220 CR-1220 Lithium Batteries DL1220,(10) Black Five Quantity DVD Empty Replacement Boxes with Wrap Around Sleeve #DV5R22BK (22mm) (5DVD),(10) Clear Quad 4-Disc Overlap Style DVD Cases / Boxes - DV4R14CL,...,vQuiet Over-Ear Noise Cancelling Headphones (Matte Black),"voxoom Low Latency aptX Bluetooth Wireless TV and Smartphone Headphones system---no lip sync or echo, good for all HD TV sets with digital audio or 3.5mm jack, compatible with all smartphones",weBoost 30' White RG6 Low Loss Coax Cable - Retail Packaging,wennow 5 Pcs F type Female to BNC Male RF Coax Connector Adapter CCTV RG6 RG59,wennow Premium 5Amp Power Adapter with 8 way splitter for Night Owl Cameras,xDuoo TA-03 XMOS WM8740 24Bit/192KHz USB DAC 12AU7 Tube Headphone Amplifier,yoomall 5M Antenna WIFI RP-SMA Female to Male Extensionl Cable,zebest okcharger902 House Wall Power Charger Cord for Barnes and Noble Nook,"{Waterproof Sports Speaker}, Wireless Mini Smart Portable Handheld Speakers Bluetooth 4.0 with 12 Hour Playtime, Shockproof IPX54, Remote Shutter Control with 7W Powerful Drive and Bass Enhance Technology for Outdoors Sports Travel Bicycle Cycling Climbing Camping Hiking Running Fishing","★ The Ultimate Bundle - 7 Pack of Premium Microfiber Cleaning Cloths ★ - Keep Your iPhone, iPad, or Any Mobile Device Clean and Clear From Smudges, Blemishes, Dust, Dirt, Oil, 100% Lint-Free - Black"
product_title,,,,,,,,,,,,,,,,,,,,,
"#1 BEST SELLING Waterproof Bluetooth Speaker with Powerbank, EVIZIAN® ROKON CHARGE 2 in 1 Ultra Portable Wireless Waterproof Shockproof Dust Proof Rugged Bluetooth Speaker with Built In Powerbank - Better Sound, Better Volume, Incredible Online Price - The Perfect Indoor Outdoor Speaker.",0.000000,17.972201,17.720045,18.000000,21.794495,17.521415,17.720045,17.972201,17.521415,17.972201,...,17.972201,18.411953,18.841444,17.378147,17.972201,17.291616,19.078784,21.213203,26.570661,20.346990
"#1 Universal TV Box Mount Holder Mounting Bracket Shelf For Apple TV 2/3, Roku TV 1/2/3, MXQ, MX 3, ONVO Matricom, SkyStreamX 4 Android TV Box, CHOETECH 6-Port, Photive 6-Port, Anker 6-Port/5-Port/2-Port, Poweradd 6-Port/5-Port Desktop USB Charger& More- Retractable Clip Design Patent Pending",17.972201,0.000000,6.403124,7.141428,14.212670,5.830952,6.403124,7.071068,5.830952,7.071068,...,7.071068,8.124038,9.055385,5.385165,7.071068,5.099020,9.539392,13.304135,20.808652,11.874342
#364 Renata Watch Batteries 2Pcs,17.720045,6.403124,0.000000,6.480741,13.892444,5.000000,5.656854,6.403124,5.000000,6.403124,...,6.403124,7.549834,8.544004,4.472136,6.403124,4.123106,9.055385,12.961481,20.591260,11.489125
#394 Renata Watch Batteries 2Pcs,18.000000,7.141428,6.480741,0.000000,14.247807,5.916080,6.480741,7.141428,5.916080,7.141428,...,7.141428,8.185353,9.110434,5.477226,7.141428,5.196152,9.591663,13.341664,20.832667,11.916375
%Original Superlux HD668B Semi-open Dynamic Stereo Professional Studio Standard Monitoring Headphones For DJ Music Detachable Audio Cable,21.794495,14.212670,13.892444,14.247807,0.000000,13.638182,13.892444,14.212670,13.63818

In [17]:
recommender_df.shape

(15961, 15961)

# Evaluate recommender performance

In [27]:
re.head(2)

,customer_id,product_title,star_rating
0,41409413,yoomall 5M Antenna WIFI RP-SMA Female to Male ...,5
1,49668221,"Hosa GPM-103 3.5mm TRS to 1/4"" TRS Adaptor",5


In [40]:
items ={}

for title in re['product_title'].unique()[:5]:


    for product_title in re.loc[re['product_title'].str.contains(title), 'product_title'].unique():
        print(product_title)
        print('Average rating', pivot.loc[product_title, :].mean())
        print('Number of ratings', pivot.T[product_title].count())
        print('')
        print('closest items')
        print(recommender_df[product_title].sort_values()[1:11])
        print('')
        print('*******************************************************************************************')
        print('')
    items[title] = recommender_df[product_title].sort_values().index.tolist()[1:11]
    


yoomall 5M Antenna WIFI RP-SMA Female to Male Extensionl Cable
Average rating 4.0
Number of ratings 4

closest items
product_title
Knox 16-Bay Rapid Ni-MH AA/AAA Battery Charger With 12 AA & 4 AAA Batteries                                                                                                                                                                            8.185353
NooElec Ham It Up v1.3 - NooElec RF Upconverter For Software Defined Radio. Works With Most SDRs Like HackRF & RTL-SDR (RTL2832U with E4000, FC0013 or R820T Tuners); MF/HF Converter With SMA Jacks                                                   8.185353
Noveske NSR Polymer Panel Set NSR-BP-BLK                                                                                                                                                                                                               8.185353
Nxg 12 500w Powered Subwoofer 24hz-200hz Black Pebble Finish                                         

In [41]:
items

{'yoomall 5M Antenna WIFI RP-SMA Female to Male Extensionl Cable': ['Knox 16-Bay Rapid Ni-MH AA/AAA Battery Charger With 12 AA & 4 AAA Batteries',
  'NooElec Ham It Up v1.3 - NooElec RF Upconverter For Software Defined Radio. Works With Most SDRs Like HackRF & RTL-SDR (RTL2832U with E4000, FC0013 or R820T Tuners); MF/HF Converter With SMA Jacks',
  'Noveske NSR Polymer Panel Set NSR-BP-BLK',
  'Nxg 12 500w Powered Subwoofer 24hz-200hz Black Pebble Finish',
  'OEM Replacement Battery Cover for Dre Beats Studio Black',
  'OEM Replacement Battery Cover for Dre Beats Studio Red Color',
  'Audio Fox Wireless TV Speakers',
  'Audio Desk Professional Grade Vinyl Cleaning Machine, Ultrasonic, Automatic, For Albums, LPs, Vinyl and 45s',
  'OLLO:30-55" Hand Brushed Aluminum Universal Table Top TV Stand, 77 Lb. Capacity, Rotation -/+45º , Tilt -5/+10º , Safety Lock/ Lock Release, Anodized Finish, Limited Edition Gold, LED, LCD (TT-01 Gold)',
  'Audio Cable For Beats with Inline Remote / Microphon

In [44]:
with open('items.json', 'w') as json_file:
    json.dump(items, json_file)
    

In [47]:
# use the above function with open() and 'read binary' permissions to get our model back
json_file = json.load(open('items.json', 'rb'))


print(json_file)

{'yoomall 5M Antenna WIFI RP-SMA Female to Male Extensionl Cable': ['Knox 16-Bay Rapid Ni-MH AA/AAA Battery Charger With 12 AA & 4 AAA Batteries', 'NooElec Ham It Up v1.3 - NooElec RF Upconverter For Software Defined Radio. Works With Most SDRs Like HackRF & RTL-SDR (RTL2832U with E4000, FC0013 or R820T Tuners); MF/HF Converter With SMA Jacks', 'Noveske NSR Polymer Panel Set NSR-BP-BLK', 'Nxg 12 500w Powered Subwoofer 24hz-200hz Black Pebble Finish', 'OEM Replacement Battery Cover for Dre Beats Studio Black', 'OEM Replacement Battery Cover for Dre Beats Studio Red Color', 'Audio Fox Wireless TV Speakers', 'Audio Desk Professional Grade Vinyl Cleaning Machine, Ultrasonic, Automatic, For Albums, LPs, Vinyl and 45s', 'OLLO:30-55" Hand Brushed Aluminum Universal Table Top TV Stand, 77 Lb. Capacity, Rotation -/+45º , Tilt -5/+10º , Safety Lock/ Lock Release, Anodized Finish, Limited Edition Gold, LED, LCD (TT-01 Gold)', 'Audio Cable For Beats with Inline Remote / Microphone for Beats by Dr.

In [ ]:
# route 4: accept the form submission and do something fancy with it
@app.route('/submit')
def submit():
    data = request.args
    suggestions = {key : info.get(key, 'Item not found') for key in data.values()}
       
    return render_template('results.html', prediction = suggestions)



<!DOCTYPE html>
<html>
<head>
	<title>Your Prediction</title>
</head>
<body>
<h2> These are other items that you will be interested in: {{prediction}}<h2>
</body>
</html>


# collaborative recommender users - user based

In [23]:
#pivotu = pd.pivot_table(re, index='customer_id', columns='product_title', values='star_rating')
#pivotu



In [24]:
#pivotu.shape

In [25]:
#sparse_pivotu = sparse.csr_matrix(pivotu.fillna(0))
#print(sparse_pivotu)


In [ ]:
#sparse_pivotu.shape

In [ ]:
#recommenderu = cosine_similarity(sparse_pivotu)


In [ ]:
#create distance dataframe
#recommenderu_df = pd.DataFrame(recommenderu, columns=pivotu.index, index=pivotu.index)
#recommenderu_df

In [ ]:
#recommenderu_df.shape

In [26]:
#evaluate recommender performance
#search = 'yaya'

#for username in re.loc[re['reviews.username'].str.contains(search), 'reviews.username']:
    print(username)
    print('Average rating', pivotu.loc[username, :].mean())
    print('Number of ratings', pivotu.T[username].count())
    print('')
    print('closest users')
    print(recommenderu_df[username].sort_values()[1:3500])
    print('')
    print('*******************************************************************************************')
    print('')

IndentationError: unexpected indent (<ipython-input-26-64c536e904ad>, line 5)